In [91]:

import psycopg2 as pg2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from datetime import datetime, date, timedelta
import importlib
from  BidTrackerScraper import BidTrackerScraper
from bs4 import BeautifulSoup
import pymongo



In [88]:
print ("Connecting to SQL")
conn = pg2.connect(user='postgres',  dbname='penny', host='localhost', port='5432', password='password')

print ("Reading Dataset")
df = pd.read_sql ("Select username, count(*) from bids group by username order by count(*) desc", conn)


Connecting to SQL
Reading Dataset


In [92]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["penny"]
pages_collection = db["pages"]


In [28]:
auction_ids

,qauctionid
0,155435135
1,100231273
2,100275412
3,100325481
4,292999047
...,...
27883,997852448
27884,998330005
27885,999003949
27886,999286635


In [3]:
bts = BidTrackerScraper()
bts.login(3)

200
200


In [127]:

for username in df.username:
    print (username)
    url = "http://www.bidtracker.info/UserAuctions?site=quibids&user="+username
    r = bts._session.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    rows = soup.findAll("tr", {"class": {"tr-winner","tr-ended"}})
    for r in rows:
        tds = r.findAll("td", {"class": "td-inner"})
        auction_id = tds[0].text
        title = tds[1].text
        if not ("Gift Card" in title or "Buster" in title or "Voucher" in title):
            continue
        if "$10 " in title:
            continue
        if "$15 " in title:
            continue
#        if "$75 Amazon Gift Card AND 10 Bids" in title:
#            continue
#        if "$100 Amazon Gift Card AND 20 Bids" in title:
#            continue
#        if "$75 Amazon Gift Card AND 20 Bids" in title:
#            continue
        cashvalue = tds[2].text
        datestr = tds[8].text
        datestr = datestr.split(" ")[0]
        if (len(datestr)==6):
            datestr += "-20"
        date = datetime.strptime(datestr, "%b-%d-%y")
        if (datetime.now()-date).days > 1 and date >= datetime(2020, 1, 1):
            #print (auction_id, int(auction_id) in set(auction_ids.qauctionid), date, title)
            item_found = pages_collection.find({"_id": auction_id},{"_id": 1}).count() > 0
            if (not item_found):
                print(auction_id,title, datestr)
    break

BidsWins2016


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


533116901 Bust One - Limit Buster! AND 30 Bids Feb-18-20
983581281 Bust One - Limit Buster! AND 30 Bids Feb-17-20
676765057 Bust One - Limit Buster! AND 30 Bids Feb-14-20
124248401 Bust One - Limit Buster! AND 30 Bids Feb-10-20
474444017 Bust One - Limit Buster! AND 30 Bids Feb-09-20
355442303 LBust One - Limit Buster! AND 30 Bids Feb-09-20
839276864  106%Bust One - Limit Buster! AND 30 Bids Feb-08-20
988485429 LBust One - Limit Buster! AND 30 Bids Feb-08-20
544404508  103%LBust One - Limit Buster! AND 30 Bids Feb-07-20
296481536  97%Bust One - Limit Buster! AND 30 Bids Feb-06-20
532843042 Bust One - Limit Buster! AND 30 Bids Feb-05-20
270450815 LBust One - Limit Buster! AND 30 Bids Feb-05-20
851201278  63%LBust One - Limit Buster! AND 30 Bids Feb-03-20
940772653  106%Bust One - Limit Buster! AND 30 Bids Jan-31-20
597158182  100%LBust One - Limit Buster! AND 30 Bids Jan-31-20
366841779  166%Bust One - Limit Buster! AND 30 Bids Jan-30-20
184200639  83%Bust One - Limit Buster! AND 30 Bid

In [68]:
int("155435135") in auction_ids.qauctionid

False

In [70]:
auction_ids.qauctionid[0] == 155435135

True

In [73]:
155435135 in pd.Series(auction_ids.qauctionid)

False

In [83]:
155435135 in set(auction_ids.qauctionid)

True